In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import spherinder as sph


In [ ]:
m, Lmax, Nmax, alpha, sigma = 1, 8, 10, 0, 0
ncoeff = Lmax*Nmax
markersize = 800
center = 's'
markers = ['d','X','P']
margin = .2

In [ ]:
# Make sure we're looking at the densest column
def max_density(op):
    maxnz, maxcol = 0, 0
    for c in range(np.shape(op)[1]):
        nzr, _ = op[:,c].nonzero()
        if len(nzr) > maxnz:
            maxnz = len(nzr)
            maxcol = c
    return maxnz, maxcol

def operator_splatter(op, Nout, Nin, ell, k):
    colindex = ell*Nin + k
    col = op[:,colindex]
    rows, _ = col.nonzero()
    maxnz, maxcol = max_density(op)
    if len(rows) != maxnz:
        raise ValueError(f'Not your densest column, {maxcol} is denser')

    inds = [(r//Nout,r%Nout) for r in rows]
    ells, ks = zip(*inds)
    ells, ks = np.asarray(ells)-ell, np.asarray(ks)-k
    return ells, ks


In [ ]:
def plot_splatter(opname, operator, codomain, ax=None, margins=margin):
    if not isinstance(operator, (list,tuple)):
        operator = [operator]
        codomain = [codomain]

    ellmin, ellmax, kmin, kmax = np.inf, -np.inf, np.inf, -np.inf
    if ax is None:
        fig, ax = plt.subplots(figsize=(3,3))
        return_fig = True
    else:
        return_fig = False

    ax.scatter(0,0,marker=center,color='tab:purple',s=markersize)
    for i, op in enumerate(operator):
        ells, ks = operator_splatter(op, codomain[i][1], Nmax, ell, k)
        ax.scatter(ks,ells,s=markersize,marker=markers[i])

        ellmin, ellmax = min(min(ells), ellmin), max(max(ells), ellmax)
        kmin, kmax = min(min(ks), kmin), max(max(ks), kmax)

    ax.set_xticks(range(int(kmin),int(kmax)+1))
    ax.set_yticks(range(int(ellmin),int(ellmax)+1))
    ax.set_aspect('equal')
    if isinstance(margins, tuple):
        ax.margins(*margins)
    else:
        ax.margins(margin)
    ax.set_xlabel('$k$')
    ax.set_ylabel('$l$')
    ax.set_title(opname)

    if return_fig:
        return fig, ax


In [ ]:
# Laplacian sparsity structure
ell, k = 4, 1
operator = sph.operator('laplacian')
codomain = operator.codomain(Lmax,Nmax,alpha)
Op = operator(m,Lmax,Nmax,alpha)

fig, ax = plot_splatter('Laplacian', Op, codomain, margins=(1.5*margin,margin))


In [ ]:
# Gradient sparsity structure
ell, k = 4, 1
operator = sph.operator('gradient')
codomain = [cd(Lmax,Nmax,alpha) for cd in operator.codomain]
Op = operator(m,Lmax,Nmax,alpha)

fig, ax = plot_splatter('Gradient', Op, codomain)


In [ ]:
# Divergence sparsity structure
ell, k = 4, 1

operator = sph.operator('divergence')
codomain = [(Lmax,Nmax+1)]*3

Op = operator(m,Lmax,Nmax,alpha)
ops = [Op[:,:ncoeff],Op[:,ncoeff:2*ncoeff],Op[:,2*ncoeff:3*ncoeff]]

fig, ax = plot_splatter('Divergence', ops, codomain)


In [ ]:
# r e_r sparsity structure
ell, k = 4, 2

for convert in [False,True]:
    operator = sph.RadialMultiplication(convert=convert)
    codomain = [cd(Lmax,Nmax,alpha) for cd in operator.codomain]
    Op = operator(m,Lmax,Nmax,alpha)

    fig, ax = plot_splatter('$r e_r$', Op, codomain)


In [ ]:
# e_r . sparsity structure
ell, k = 4, 1

operator = sph.operator('rdot')
codomain = [(Lmax+1,Nmax+1)]*3
Op = operator(m,Lmax,Nmax,alpha)
ops = [Op[:,:ncoeff],Op[:,ncoeff:2*ncoeff],Op[:,2*ncoeff:3*ncoeff]]

fig, ax = plot_splatter('$e_r \cdot$', ops, codomain)


In [ ]:
# 1-r**2 sparsity structure
ell, k = 4, 1
operator = sph.operator('1-r**2')
codomain = operator.codomain(Lmax,Nmax,alpha)
Op = operator(m,Lmax,Nmax,alpha=1,sigma=0)

fig, ax = plot_splatter('$1-r^2$', Op, codomain)


In [ ]:
# conversion sparsity structure
ell, k = 4, 1
operator = sph.operator('conversion')
codomain = operator.codomain(Lmax,Nmax,alpha)
Op = operator(m,Lmax,Nmax,alpha=0,sigma=0)

fig, ax = plot_splatter('$\\alpha$-conversion', Op, codomain)


In [ ]:
# Differential operators
fig, ax = plt.subplots(1,3,figsize=(10,3))

# Gradient sparsity structure
ell, k = 4, 2
operator = sph.operator('gradient')
codomain = [cd(Lmax,Nmax,alpha) for cd in operator.codomain]
Op = operator(m,Lmax,Nmax,alpha)
plot_splatter('Gradient', Op, codomain, ax=ax[0])

# Divergence sparsity structure
ell, k = 4, 2
operator = sph.operator('divergence')
codomain = [(Lmax,Nmax+1)]*3
Op = operator(m,Lmax,Nmax,alpha)
ops = [Op[:,:ncoeff],Op[:,ncoeff:2*ncoeff],Op[:,2*ncoeff:3*ncoeff]]
plot_splatter('Divergence', ops, codomain, ax=ax[1])

# Laplacian sparsity structure
ell, k = 4, 2
operator = sph.operator('laplacian')
codomain = operator.codomain(Lmax,Nmax,alpha)
Op = operator(m,Lmax,Nmax,alpha)
plot_splatter('Laplacian', Op, codomain, ax=ax[2], margins=(1.5*margin,margin))


In [ ]:
# radial operators
fig, ax = plt.subplots(1,4,figsize=(16,3))

# r e_r sparsity structure
ell, k = 4, 2
operator = sph.RadialMultiplication(convert=False)
codomain = [cd(Lmax,Nmax,alpha) for cd in operator.codomain]
Op = operator(m,Lmax,Nmax,alpha)
plot_splatter('$r$ $e_r$', Op, codomain, ax=ax[0])

# e_r . sparsity structure
ell, k = 4, 2
operator = sph.operator('rdot')
codomain = [(Lmax+1,Nmax+1)]*3
Op = operator(m,Lmax,Nmax,alpha)
ops = [Op[:,:ncoeff],Op[:,ncoeff:2*ncoeff],Op[:,2*ncoeff:3*ncoeff]]
plot_splatter('$e_r \cdot$', ops, codomain, ax=ax[1])

# 1-r**2 sparsity structure
ell, k = 4, 2
operator = sph.operator('1-r**2')
codomain = operator.codomain(Lmax,Nmax,alpha)
Op = operator(m,Lmax,Nmax,alpha=1,sigma=0)
plot_splatter('$1-r^2$', Op, codomain, ax=ax[2])

# conversion sparsity structure
ell, k = 4, 2
operator = sph.operator('conversion')
codomain = operator.codomain(Lmax,Nmax,alpha)
Op = operator(m,Lmax,Nmax,alpha=0,sigma=0)
plot_splatter('$\\alpha$-conversion', Op, codomain, ax=ax[3])
